# Gram-schmidt and the KF

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
%config InlineBackend.figure_format = "retina"

In [5]:
np.random.seed(314)

a = 3
y = np.random.randn(100)
e = np.random.normal(scale=0.1, size=100)

f = a * y + e

In [6]:
# sample covariance

var_y = np.var(y, ddof=0)

In [7]:
cov = np.cov(f, y, ddof=0)
cov

array([[8.06388127, 2.66652296],
       [2.66652296, 0.88301421]])

In [8]:
cov_fy = cov[0, 1]
var_y = cov[1, 1]

In [12]:
# Best linear unbiased predictor
cov_fy / var_y

3.0197961909519537

## Gram-shmidt

In [34]:
np.cov(f[2], y[0], ddof=0)

array([[0., 0.],
       [0., 0.]])

## References
- [Covariance numpy](https://stackoverflow.com/questions/15317822/calculating-covariance-with-python-and-numpy)